<a href="https://colab.research.google.com/github/nbht98/attack_cat_or_dog_classification/blob/master/TEST_CIFAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [209]:
!pip3 install mia

In [0]:
# Import library 
import numpy as np

from absl import app
from absl import flags

import tensorflow as tf
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

from mia.estimators import ShadowModelBundle, AttackModelBundle, prepare_attack_data


In [0]:
NUM_CLASSES = 10
WIDTH = 32
HEIGHT = 32
CHANNELS = 3

TARGET_EPOCHS = 100 # Number of epochs to train target and shadow models
ATTACK_EPOCHS = 100 # Number of epochs to train attack models.
NUM_SHADOWS = 3 # Number of epochs to train attack models.

In [0]:
def get_data():
    """Prepare CIFAR10 data."""
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
    y_train = tf.keras.utils.to_categorical(y_train)
    y_test = tf.keras.utils.to_categorical(y_test)
    X_train = X_train.astype("float32")
    X_test = X_test.astype("float32")
    y_train = y_train.astype("float32")
    y_test = y_test.astype("float32")
    X_train /= 255
    X_test /= 255
    return (X_train, y_train), (X_test, y_test)


In [0]:
def target_model_fn():
  """The architecture of the target (victim) model.
  The attack is white-box, hence the attacker is assumed to know this architecture too."""

  model = tf.keras.models.Sequential()

  model.add(
      layers.Conv2D(
          32,
          (3, 3),
          activation="relu",
          padding="same",
          input_shape=(WIDTH, HEIGHT, CHANNELS),
      )
  )
  model.add(layers.Conv2D(32, (3, 3), activation="relu"))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Dropout(0.25))

  model.add(layers.Conv2D(64, (3, 3), activation="relu", padding="same"))
  model.add(layers.Conv2D(64, (3, 3), activation="relu"))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Dropout(0.25))

  model.add(layers.Flatten())

  model.add(layers.Dense(512, activation="relu"))
  model.add(layers.Dropout(0.5))

  model.add(layers.Dense(NUM_CLASSES, activation="softmax"))
  model.compile("adam", loss="categorical_crossentropy", metrics=["accuracy"])

  return model


def attack_model_fn():
  """Attack model that takes target model predictions and predicts membership.
  Following the original paper, this attack model is specific to the class of the input.
  AttachModelBundle creates multiple instances of this model for each class.
  """
  model = tf.keras.models.Sequential()

  model.add(layers.Dense(128, activation="relu", input_shape=(NUM_CLASSES,)))

  model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
  model.add(layers.Dense(64, activation="relu"))
  model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
  model.add(layers.Dense(64, activation="relu"))

  model.add(layers.Dense(1, activation="sigmoid"))
  model.compile("adam", loss="binary_crossentropy", metrics=["accuracy"])
  return model


In [0]:
(X_train, y_train), (X_test, y_test) = get_data()

In [215]:
# Train the target model.
print("Training the target model...")
target_model = target_model_fn()
target_model.fit(
    X_train, y_train, epochs=TARGET_EPOCHS, validation_split=0.1, verbose=True
)

Training the target model...
Epoch 1/100
1407/1407 [==============================] - 9s 6ms/step - loss: 1.5504 - accuracy: 0.4306 - val_loss: 1.1340 - val_accuracy: 0.5898
Epoch 2/100
1407/1407 [==============================] - 8s 6ms/step - loss: 1.1649 - accuracy: 0.5830 - val_loss: 1.0453 - val_accuracy: 0.6402
Epoch 3/100
1407/1407 [==============================] - 8s 6ms/step - loss: 1.0145 - accuracy: 0.6418 - val_loss: 0.9090 - val_accuracy: 0.6818
Epoch 4/100
1407/1407 [==============================] - 8s 6ms/step - loss: 0.9172 - accuracy: 0.6761 - val_loss: 0.7939 - val_accuracy: 0.7238
Epoch 5/100
1407/1407 [==============================] - 8s 6ms/step - loss: 0.8530 - accuracy: 0.7010 - val_loss: 0.7702 - val_accuracy: 0.7362
Epoch 6/100
1407/1407 [==============================] - 8s 6ms/step - loss: 0.7999 - accuracy: 0.7181 - val_loss: 0.7510 - val_accuracy: 0.7406
Epoch 7/100
1407/1407 [==============================] - 8s 6ms/step - loss: 0.7532 - accuracy: 0.736

In [216]:
# We assume that attacker's data were not seen in target's training.
attacker_X_train, attacker_X_test, attacker_y_train, attacker_y_test = train_test_split(
    X_test, y_test, test_size=0.1
)
print(attacker_X_train.shape, attacker_X_test.shape)

(9000, 32, 32, 3) (1000, 32, 32, 3)


In [0]:
SHADOW_DATASET_SIZE = 4000

# Train the shadow models.
smb = ShadowModelBundle(
    target_model_fn,
    shadow_dataset_size=SHADOW_DATASET_SIZE,
    num_models=NUM_SHADOWS,
)

In [218]:
print("Training the shadow models...")
X_shadow, y_shadow = smb.fit_transform(
    attacker_X_train,
    attacker_y_train,
    fit_kwargs=dict(
        epochs=TARGET_EPOCHS,
        verbose=True,
        validation_data=(attacker_X_test, attacker_y_test),
    ),
)


Training the shadow models...
Epoch 1/100
125/125 [==============================] - 1s 7ms/step - loss: 2.1605 - accuracy: 0.1873 - val_loss: 1.9494 - val_accuracy: 0.2820
Epoch 2/100
125/125 [==============================] - 1s 6ms/step - loss: 1.8605 - accuracy: 0.3215 - val_loss: 1.7280 - val_accuracy: 0.3580
Epoch 3/100
125/125 [==============================] - 1s 6ms/step - loss: 1.6928 - accuracy: 0.3772 - val_loss: 1.5986 - val_accuracy: 0.4230
Epoch 4/100
125/125 [==============================] - 1s 6ms/step - loss: 1.5773 - accuracy: 0.4295 - val_loss: 1.5527 - val_accuracy: 0.4430
Epoch 5/100
125/125 [==============================] - 1s 6ms/step - loss: 1.4793 - accuracy: 0.4627 - val_loss: 1.4873 - val_accuracy: 0.4500
Epoch 6/100
125/125 [==============================] - 1s 6ms/step - loss: 1.3824 - accuracy: 0.4942 - val_loss: 1.4086 - val_accuracy: 0.4970
Epoch 7/100
125/125 [==============================] - 1s 6ms/step - loss: 1.2868 - accuracy: 0.5353 - val_loss:

In [0]:
# ShadowModelBundle returns data in the format suitable for the AttackModelBundle.
amb = AttackModelBundle(attack_model_fn, num_classes=NUM_CLASSES)

In [220]:
# Fit the attack models.
print("Training the attack models...")
amb.fit(
    X_shadow, y_shadow, fit_kwargs=dict(epochs=ATTACK_EPOCHS, verbose=True)
)

Training the attack models...
Epoch 1/100
76/76 [==============================] - 0s 2ms/step - loss: 0.6050 - accuracy: 0.6828
Epoch 2/100
76/76 [==============================] - 0s 2ms/step - loss: 0.5158 - accuracy: 0.7326
Epoch 3/100
76/76 [==============================] - 0s 2ms/step - loss: 0.5006 - accuracy: 0.7351
Epoch 4/100
76/76 [==============================] - 0s 2ms/step - loss: 0.4900 - accuracy: 0.7528
Epoch 5/100
76/76 [==============================] - 0s 3ms/step - loss: 0.4816 - accuracy: 0.7622
Epoch 6/100
76/76 [==============================] - 0s 3ms/step - loss: 0.4690 - accuracy: 0.7664
Epoch 7/100
76/76 [==============================] - 0s 2ms/step - loss: 0.4647 - accuracy: 0.7684
Epoch 8/100
76/76 [==============================] - 0s 2ms/step - loss: 0.4652 - accuracy: 0.7733
Epoch 9/100
76/76 [==============================] - 0s 2ms/step - loss: 0.4579 - accuracy: 0.7787
Epoch 10/100
76/76 [==============================] - 0s 2ms/step - loss: 0.451

In [0]:
def get_data_shadow():
    """Prepare CIFAR100 data."""
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar100.load_data()
    y_train = tf.keras.utils.to_categorical(y_train)
    y_test = tf.keras.utils.to_categorical(y_test)
    X_train = X_train.astype("float32")
    X_test = X_test.astype("float32")
    y_train = y_train.astype("float32")
    y_test = y_test.astype("float32")
    X_train /= 255
    X_test /= 255
    return (X_train, y_train), (X_test, y_test)

(attacker_X, attacker_y), (attacker_X_t, attacker_y_t) = get_data_shadow()

In [222]:
delete_train = []
delete_test = []
for i in range(len(attacker_y)):
  if(not np.any(attacker_y[i][30:40])):
    delete_train.append(i)

for i in range(len(attacker_y_t)):
  if(not np.any(attacker_y_t[i][30:40])):
    delete_test.append(i)

a_X_train = np.delete(attacker_X, delete, 0)
a_y_train = np.delete(attacker_y, delete, 0)
a_y_train = np.delete(a_y_train,slice(40,100), 1)
a_y_train = np.delete(a_y_train,slice(0,30), 1)
a_X_test = np.delete(attacker_X_t, delete, 0)
a_y_test = np.delete(attacker_y_t, delete, 0)
a_y_test = np.delete(a_y_test,slice(40,100), 1)
a_y_test = np.delete(a_y_test,slice(0,30), 1)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  app.launch_new_instance()


In [223]:
SHADOW_DATASET_SIZE_O = 2000

# Train the shadow models.
smb_o = ShadowModelBundle(
    target_model_fn,
    shadow_dataset_size=SHADOW_DATASET_SIZE_O,
    num_models=NUM_SHADOWS,
)
print("Training the shadow models...")
X_shadow_o, y_shadow_o = smb_o.fit_transform(
    a_X_train,
    a_y_train,
    fit_kwargs=dict(
        epochs=TARGET_EPOCHS,
        verbose=True,
        validation_data=(a_X_test, a_y_test),
    ),
)


Training the shadow models...
Epoch 1/100
63/63 [==============================] - 1s 10ms/step - loss: 2.2158 - accuracy: 0.1600 - val_loss: 0.1865 - val_accuracy: 0.1110
Epoch 2/100
63/63 [==============================] - 0s 7ms/step - loss: 1.9216 - accuracy: 0.3170 - val_loss: 0.1644 - val_accuracy: 0.1070
Epoch 3/100
63/63 [==============================] - 0s 8ms/step - loss: 1.7610 - accuracy: 0.3820 - val_loss: 0.1488 - val_accuracy: 0.1120
Epoch 4/100
63/63 [==============================] - 0s 8ms/step - loss: 1.6777 - accuracy: 0.4160 - val_loss: 0.1486 - val_accuracy: 0.1160
Epoch 5/100
63/63 [==============================] - 0s 8ms/step - loss: 1.5826 - accuracy: 0.4565 - val_loss: 0.1450 - val_accuracy: 0.1090
Epoch 6/100
63/63 [==============================] - 0s 7ms/step - loss: 1.5147 - accuracy: 0.4770 - val_loss: 0.1367 - val_accuracy: 0.1181
Epoch 7/100
63/63 [==============================] - 0s 7ms/step - loss: 1.4254 - accuracy: 0.5215 - val_loss: 0.1482 - val

In [224]:
# ShadowModelBundle returns data in the format suitable for the AttackModelBundle.
amb_o = AttackModelBundle(attack_model_fn, num_classes=NUM_CLASSES)

# Fit the attack models.
print("Training the attack models...")
amb_o.fit(
    X_shadow_o, y_shadow_o, fit_kwargs=dict(epochs=ATTACK_EPOCHS, verbose=True)
)

Training the attack models...
Epoch 1/100
37/37 [==============================] - 0s 2ms/step - loss: 0.6816 - accuracy: 0.5330
Epoch 2/100
37/37 [==============================] - 0s 2ms/step - loss: 0.6312 - accuracy: 0.6066
Epoch 3/100
37/37 [==============================] - 0s 2ms/step - loss: 0.5992 - accuracy: 0.6371
Epoch 4/100
37/37 [==============================] - 0s 2ms/step - loss: 0.5947 - accuracy: 0.6362
Epoch 5/100
37/37 [==============================] - 0s 2ms/step - loss: 0.5960 - accuracy: 0.6337
Epoch 6/100
37/37 [==============================] - 0s 2ms/step - loss: 0.5889 - accuracy: 0.6404
Epoch 7/100
37/37 [==============================] - 0s 2ms/step - loss: 0.5869 - accuracy: 0.6421
Epoch 8/100
37/37 [==============================] - 0s 2ms/step - loss: 0.5876 - accuracy: 0.6371
Epoch 9/100
37/37 [==============================] - 0s 2ms/step - loss: 0.5871 - accuracy: 0.6396
Epoch 10/100
37/37 [==============================] - 0s 3ms/step - loss: 0.582

In [226]:
# Test the success of the attack.
ATTACK_TEST_DATASET_SIZE = 4000
# Prepare examples that were in the training, and out of the training.
data_in = X_train[:ATTACK_TEST_DATASET_SIZE], y_train[:ATTACK_TEST_DATASET_SIZE]
data_out = X_test[:ATTACK_TEST_DATASET_SIZE], y_test[:ATTACK_TEST_DATASET_SIZE]
# data_out = np.concatenate((X_test, a_X_train), axis=0), np.concatenate((y_test, a_y_train), axis=0)


# Compile them into the expected format for the AttackModelBundle.
attack_test_data, real_membership_labels = prepare_attack_data(
    target_model, data_in, data_out
)

# Compute the attack accuracy.
attack_guesses = amb.predict(attack_test_data)
attack_accuracy = np.mean(attack_guesses == real_membership_labels)


attack_guesses_o = amb_o.predict(attack_test_data)
attack_accuracy_o = np.mean(attack_guesses_o == real_membership_labels)

print(attack_accuracy)
print(attack_accuracy_o)

0.596375
0.62
